## Imports

In [2]:
!pip install transformers
!pip install unidecode
!pip install sentencepiece

  Using cached transformers-4.23.1-py3-none-any.whl (5.3 MB)
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
  Using cached tokenizers-0.13.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.6 MB)

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
  Using cached sentencepiece-0.1.97-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, Trainer, TrainingArguments, AdamW
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from scripts.model_utils import transform_label, truncate, encode_tags, QuintenDataset, evaluate_model, predict_test

In [5]:
SENTENCES_SIZE = 128
TOKENS_NUMBER = 256

### Export des données augmentées

In [6]:
from scripts.doccano_to_camembert import JsonlDoccano as jd
file = jd("""scripts/data/labelled_trainset.jsonl""", camembert=True, labels_augmentation=2, back_translation=True)
#file.run_export("scripts/data/tokens_v5.parquet")
df = file.df_augmented
df.label = df.label.apply(lambda x: transform_label(x)) # Deleting B- I- tags
df.T

Token indices sequence length is longer than the specified maximum sequence length for this model (794 > 512). Running this sequence through the model will result in indexing errors


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,56235,56236,56237,56238,56239,56240,56241,56242,56243,56244,56245,56246,56247,56248,56249,56250,56251,56252,56253,56254,56255,56256,56257,56258,56259,56260,56261,56262,56263,56264,56265,56266,56267,56268,56269,56270,56271,56272,56273,56274,56275,56276,56277,56278,56279,56280,56281,56282,56283,56284,56285,56286,56287,56288,56289,56290,56291,56292,56293,56294,56295,56296,56297,56298,56299,56300,56301,56302,56303,56304,56305,56306,56307,56308,56309,56310,56311,56312,56313,56314,56315,56316,56317,56318,56319,56320,56321,56322,56323,56324,56325,56326,56327,56328,56329,56330,56331,56332,56333,56334,56335,56336,56337,56338,56339,56340,56341,56342,56343,56344,56345,56346,56347,56348,56349,56350,56351,56352,56353,56354,56355,56356,56357,56358,56359,56360,56361,56362,56363,56364,56365,56366,56367,56368,56369,56370,56371,56372,56373,56374,56375,56376,56377,56378,56379,56380,56381,56382,56383,56384,56385,56386,56387,56388,56389,56390,56391,56392,56393,56394,56395,56396,56397,56398,56399,56400,56401,56402,56403,56404,56405,56406,56407,56408,56409,56410,56411,56412,56413,56414,56415,56416,56417,56418,56419,56420,56421,56422,56423,56424,56425,56426,56427,56428,56429,56430,56431,56432,56433,56434,56435,56436,56437,56438,56439,56440,56441,56442,56443,56444,56445,56446,56447,56448,56449,56450,56451,56452,56453,56454,56455,56456,56457,56458,56459,56460,56461,56462,56463,56464,56465,56466,56467,56468,56469,56470,56471,56472,56473,56474,56475,56476,56477,56478,56479,56480,56481,56482,56483,56484
token,▁Un,▁homme,▁âgé,▁de,▁77,▁ans,▁présentait,▁des,▁signes,▁cliniques,▁de,▁pro,stat,isme,.,▁Le,▁TR,▁not,ait,▁une,▁masse,▁ré,ni,tente,.,▁Le,▁taux,▁de,▁PSA,▁était,▁de,▁,"9,5",▁n,g,/,ml,.,▁L,',E,ER,▁et,▁l,',imagerie,▁par,▁résonance,▁magnétique,▁nucléaire,▁(,IRM,),▁de,▁la,▁prostate,▁mont,raient,▁dans,▁la,▁Z,P,▁droite,",",▁une,▁masse,▁,ky,s,tique,▁à,▁parois,▁épaisse,s,▁de,▁4,cm,▁de,▁diamètre,▁et,▁contenant,▁des,▁végétation,s,▁en,dol,umi,nale,s,.,▁L,',analyse,▁,histo,logique,▁des,▁bio,ps,ies,▁pro,statique,s,▁écho,gu,idées,▁montrait,▁un,▁ad,éno,car,ci,nome,▁duc,tal,▁de,▁score,▁de,▁G,le,as,on,▁7,▁(4,+,3),▁et,▁de,▁contingent,▁tra,bé,culaire,▁pré,dom,in,ant,.,▁L,',analyse,▁immuno,-,histo,logique,▁pour,▁les,▁cyto,k,érat,ines,▁de,▁haut,▁poids,▁moléculaire,▁(,anti,▁C,K,▁90,3),▁était,▁négative,.,▁Le,▁stade,▁clinique,▁était,▁lors,▁du,▁diagnostic,▁:,▁T,3,N,x,M,0,.,▁Un,▁traitement,▁par,▁blocage,▁,andro,génique,▁complet,▁a,▁été,▁indiqué,.,▁Le,▁PSA,▁à,▁6,▁mois,▁était,▁à,"▁0,3",6,▁n,g,/,ml,▁puis,▁avait,▁augmenté,▁progressivement,▁jusqu,',à,▁atteindre,▁3,",",2,▁n,g,/,ml,",",▁3,▁ans,▁après,▁le,▁diagnostic,.,▁Une,▁E,ER,▁de,▁contrôle,▁à,▁3,▁ans,▁a,▁été,▁réalisée,▁et,▁montrait,▁la,▁persistance,▁de,▁cette,▁masse,▁,ky,s,tique,▁mais,▁avec,▁des,...,▁en,▁place,▁associé,▁à,▁une,▁en,té,roc,y,st,oplastie,▁d,',élargissement,▁en,▁raison,▁de,▁l,',hypo,can,ie,▁sévère,.,▁Le,▁patient,▁a,▁continué,auto,-,son,dage,s,.,▁Ils,▁sont,▁un,▁patient,▁de,▁59,▁ans,▁qui,▁s,',est,▁présenté,▁aux,▁urgences,▁à,▁une,▁,hémat,urie,▁macro,scopique,▁spontanée,",",▁indo,lore,▁sans,▁fièvre,▁ni,▁perte,▁de,▁poids,.,L,',examen,▁était,▁normal,

## Prétraitement et tokenization

In [7]:
df = df.groupby('id_phrase').agg(lambda x: list(x)).reset_index()
df.columns = ["id_phrase", "tokens", "ner_tags"]
df.head(3)

,id_phrase,tokens,ner_tags
0,1000,"[▁Un, ▁homme, ▁âgé, ▁de, ▁77, ▁ans, ▁présentai...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,2000,"[▁Mr, ., ▁R, ,, R, ▁âgé, ▁de, ▁53, ▁ans, ▁sans...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,2001,"[▁Le, ▁patient, ▁a, ▁été, ▁revu, ▁1, ▁mois, ▁a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [8]:
df = truncate(df) # Toutes les listes de plus de 128 éléments sont dubdivisées en plusieurs listes

In [9]:
tokens = df["tokens"].to_list()
ner_tags = df["ner_tags"].to_list()

In [10]:
#Train test split
train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens, ner_tags, test_size=0.1)

In [11]:
unique_tags = ['O', 'Dosage', 'Drug', 
'Duration', 'Form', 'Frequency', 'Route', 'Treatment']

tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [12]:
#To keep BIO tags
"""#unique_tags = set(tag for doc in ner_tags for tag in doc)
unique_tags = ['O', 'B-Dosage', 'I-Dosage', 'B-Drug', 'I-Drug', 
'B-Duration', 'I-Duration', 'B-Form', 'I-Form', 'B-Frequency', 
'I-Frequency', 'B-Route', 'I-Route', 'B-Treatment', 'I-Treatment']

tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}
"""

"#unique_tags = set(tag for doc in ner_tags for tag in doc)\nunique_tags = ['O', 'B-Dosage', 'I-Dosage', 'B-Drug', 'I-Drug', \n'B-Duration', 'I-Duration', 'B-Form', 'I-Form', 'B-Frequency', \n'I-Frequency', 'B-Route', 'I-Route', 'B-Treatment', 'I-Treatment']\n\ntag2id = {tag: id for id, tag in enumerate(unique_tags)}\nid2tag = {id: tag for tag, id in tag2id.items()}\n"

In [13]:
#Tokenization
tokenizer = CamembertTokenizerFast.from_pretrained('camembert-base')
train_encodings = tokenizer(train_tokens, is_split_into_words=True, return_offsets_mapping=True, padding='max_length',
                            truncation='longest_first',
                            max_length=TOKENS_NUMBER)
val_encodings = tokenizer(val_tokens, is_split_into_words=True, return_offsets_mapping=True, padding='max_length',
                            truncation='longest_first',
                            max_length=TOKENS_NUMBER)

train_labels = encode_tags(train_tags, train_encodings, tag2id)
val_labels = encode_tags(val_tags, val_encodings, tag2id)

In [14]:
train_encodings.pop("offset_mapping") 
val_encodings.pop("offset_mapping")

train_dataset = QuintenDataset(train_encodings, train_labels)
val_dataset = QuintenDataset(val_encodings, val_labels)

## Train model

In [22]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [23]:
model = CamembertForTokenClassification.from_pretrained('camembert-base', 
                                                        num_labels=len(unique_tags), 
                                                        id2label=id2tag, 
                                                        label2id=tag2id).to(device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForTokenClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream tas

In [103]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in tqdm(range(20)):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

100%|██████████| 20/20 [38:58<00:00, 116.95s/it]


CamembertForTokenClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0): CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
        

In [102]:
torch.save(model, 'model_v12.pb')

## Evaluation

In [18]:
evaluate_model('model_v11.pb', val_dataset, val_labels, id2tag)

Sur l'ensemble des vrais labels différents de O et des labels prédits différents de O, 29.5% ont le bon label prédit


## Predict 

In [153]:
model = torch.load('model_v11.pb')

In [32]:
from scripts.csv_and_doccano import CsvDoccano as cd
csv = cd('scripts/data/test.csv')
df_test = csv.df
df = df_test.groupby('sentence_id').agg(lambda x: list(x)).reset_index()
df.columns = ["id_phrase", "ner_tags", "tokens"]
df = truncate(df)

tokens = df["tokens"].to_list()
tokenizer = CamembertTokenizerFast.from_pretrained('camembert-base')

df = predict_test(df, model, id2tag, tokenizer)

,286,287,288,289,290,291,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,317,318,523,524,611,613,614,615,689,690,694,776,777,778,782,783,784,1369,1370,1371,1372,1373,1374,1375,1376,1377,1547,1548,1549,1550,1571,1572,1573,1615,1616,1617,1618,1640,1641,1642,1707,1708,1709,3087,3088,3089,3091,3092,3093,3094,3095,3096,3113,3114,3115,3116,3117,3118,3119,3120,3124,3125,3126,3127,3128,3129,3130,3131,3132,3133,3134,3135,3136,3137,3138,3139,3141,3142,3143,3144,3145,3146,3147,3148,3149,3150,3151,3152,3153,3154,3156,3157,3158,3159,3160,3161,3162,3163,3164,3165,3166,3167,3168,3169,3186,3187,3190,3191,3192,3193,3194,3195,3197,3198,3199,3200,3201,3202,3204,3205,3206,3207,3208,3209,3220,3221,3222,3223,3224,3225,3236,3237,3238,3239,3243,3244,3245,3246,3247,3248,3334,3335,3336,3337,3338,3339,3347,3348,3349,3350,3351,3352,3353,3354,3355,3356,3357,3358,3359,3361,3362,3363,3364,3365,3366,3367,3368,3372,3373,3374,3375,3376,3377,3379,3380,3381,3382,3383,3384,3385,3386,3387,3395,3396,3397,3398,3399,3400,3401,3402,3403,3404,3405,3406,3407,3408,3409,3422,3423,3424,3425,3426,3427,3429,3430,3431,3432,3433,3434,3435,3436,3437,3438,3439,3440,3441,3442,3445,3446,3447,3448,3449,3450,3451,3452,3453,3454,...,60058,60070,60071,60072,60073,60074,60075,60076,60077,60078,60080,60436,60948,60949,60950,60951,60952,61365,61366,61367,61368,61369,61370,61371,61372,61373,61374,61375,61376,61377,61378,61379,61380,61381,61382,61383,61384,61385,61386,61387,61388,61389,61390,61391,61392,61393,61394,61395,61396,62047,62048,62049,62050,62051,62052,62053,62054,62055,62056,62057,62058,62059,62060,62061,62062,62063,62064,62065,62066,62067,62068,62069,62070,62071,62072,62073,62074,62075,62300,62301,62302,62303,62304,62305,62306,62307,62308,62309,62310,62311,62316,62892,62893,63086,63087,63088,63089,63090,63091,63092,63093,63674,63675,63676,63677,63678,63679,63680,63745,63746,63747,63748,63749,63750,63751,63752,63753,63754,64363,64364,64365,64366,64367,64368,64369,64370,64371,64372,64373,64374,64375,64376,64377,64838,64839,64840,64841,64904,64905,64906,64907,64908,66241,66242,66243,66244,66627,66628,66629,66630,66631,66638,66639,66640,66980,66981,66982,66983,66984,66985,66986,66987,66988,66989,66990,67600,67601,67602,67603,67604,67609,67610,67611,67612,67614,67654,67976,67977,67978,67979,67980,67981,67982,67983,68168,68169,68170,68171,68172,68173,68783,68784,68785,68953,68954,68955,68956,68957,68958,68959,68960,68961,68962,68979,68980,68981,68985,68986,68987,68988,68989,68996,69984,69985,69989,69990,69991,69992,69993,69994,69995,69996,69997,69998,70001,70002,70003,70004,70005,70016,70017,70018,70019,70020,70021,70022,70488,70489,70517,70518,70519,70536,70537,70784,70785,70786,70787,70788,70789,70790
token,▁traitement,▁pro,phyl,ac,tique,▁par,▁,rif,amp,ici,ne,▁-,▁iso,nia,z,ide,▁(,▁600,▁-,▁300,▁mg,▁par,▁jour,▁trois,▁mois,chi,anesthésie,▁1,▁soit,▁0.,1,▁10,▁mg,▁1,▁(,▁1,▁mg,▁0.,1,▁mg,▁s,éd,ation,▁par,▁,ox,az,ép,am,▁CL,HP,UV,▁-,▁CP,G,▁-,▁C,LU,HP,▁-,▁CL,HP,▁-,▁de,▁chlor,hydrate,▁traitement,▁de,▁l,▁,œ,d,ème,▁secondaire,▁à,▁de,x,a,méth,as,one,▁,",",▁4,▁mg,▁en,▁intra,ve,in,euse,▁toutes,▁les,▁six,▁heures,▁pendant,▁24,▁heures,▁,",",▁2,▁mg,▁par,▁voie,▁orale,▁toutes,▁les,▁six,▁heures,▁pendant,▁24,▁heures,▁,",",▁1,▁mg,▁par,▁voie,▁orale,▁toutes,▁les,▁six,▁heures,▁pendant,▁24,▁heures,▁,.,▁oral,▁à,▁pyr,i,méth,amine,▁,",",’,▁acide,▁,foli,nique,▁et,▁s,ulf,a,dia,zine,▁a,▁pyr,i,méth,amine,▁,",",▁200,▁mg,▁,",",▁quotidienne,▁de,▁75,▁mg,▁,.,▁acide,▁,foli,nique,▁,",",▁50,▁mg,▁une,▁fois,▁par,▁jour,▁jusqu,’,▁au,▁jour,▁6,▁,",",▁15,▁mg,▁une,▁fois,▁par,▁jour,▁,.,▁s,ulf,a,dia,zine,▁était,▁1,▁500,▁mg,▁quatre,▁fois,▁par,▁jour,▁,.,▁traitement,▁à,▁l,',▁infection,▁To,xo,pla,s,ma,▁,gond,ii,▁,.,▁traitement,▁oral,▁contre,▁le,▁VIH,▁constitué,▁t,éno,fo,vir,▁(,▁300,▁mg,▁une,▁fois,▁par,▁jour,▁),▁,",",▁em,tric,ita,bine,▁(,▁200,▁mg,▁une,▁fois,▁par,...,▁en,▁né,phro,s,tom,ie,▁,percu,tan,ée,gu,▁chimiothérapie,▁né,ph,r,ectomie,▁élargie,▁chimiothérapie,▁a,dj,uv,ante,▁de,▁type,▁(,▁cyclo,pho,s,pha,mid,e,▁,",",▁ad

In [33]:
result_final.reset_index().to_csv('scripts/data/prediction.csv')

from scripts.pred_to_submit import Prediction as pred

test = pred('scripts/data/test.csv', 'scripts/data/prediction.csv')
df = test.to_submission()

# File to process with regex
test.submission.to_csv('scripts/prediction_before_regex.csv')
df.T

100%|██████████| 60/60 [00:18<00:00,  3.29it/s]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,32307,32308,32309,32310,32311,32312,32313,32314,32315,32316,32317,32318,32319,32320,32321,32322,32323,32324,32325,32326,32327,32328,32329,32330,32331,32332,32333,32334,32335,32336,32337,32338,32339,32340,32341,32342,32343,32344,32345,32346,32347,32348,32349,32350,32351,32352,32353,32354,32355,32356,32357,32358,32359,32360,32361,32362,32363,32364,32365,32366,32367,32368,32369,32370,32371,32372,32373,32374,32375,32376,32377,32378,32379,32380,32381,32382,32383,32384,32385,32386,32387,32388,32389,32390,32391,32392,32393,32394,32395,32396,32397,32398,32399,32400,32401,32402,32403,32404,32405,32406,32407,32408,32409,32410,32411,32412,32413,32414,32415,32416,32417,32418,32419,32420,32421,32422,32423,32424,32425,32426,32427,32428,32429,32430,32431,32432,32433,32434,32435,32436,32437,32438,32439,32440,32441,32442,32443,32444,32445,32446,32447,32448,32449,32450,32451,32452,32453,32454,32455,32456,32457,32458,32459,32460,32461,32462,32463,32464,32465,32466,32467,32468,32469,32470,32471,32472,32473,32474,32475,32476,32477,32478,32479,32480,32481,32482,32483,32484,32485,32486,32487,32488,32489,32490,32491,32492,32493,32494,32495,32496,32497,32498,32499,32500,32501,32502,32503,32504,32505,32506,32507,32508,32509,32510,32511,32512,32513,32514,32515,32516,32517,32518,32519,32520,32521,32522,32523,32524,32525,32526,32527,32528,32529,32530,32531,32532,32533,32534,32535,32536,32537,32538,32539,32540,32541,32542,32543,32544,32545,32546,32547,32548,32549,32550,32551,32552,32553,32554,32555,32556
TokenId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,32307,32308,32309,32310,32311,32312,32313,32314,32315,32316,32317,32318,32319,32320,32321,32322,32323,32324,32325,32326,32327,32328,32329,32330,32331,32332,32333,32334,32335,32336,32337,32338,32339,32340,32341,32342,32343,32344,32345,32346,32347,32348,32349,32350,32351,32352,32353,32354,32355,32356,32357,32358,32359,32360,32361,32362,32363,32364,32365,32366,32367,32368,32369,32370,32371,32372,32373,32374,32375,32376,32377,32378,32379,32380,32381,32382,32383,32384,32385,32386,32387,32388,32389,32390,32391,32392,32393,32394,32395,32396,32397,32398,32399,32400,32401,32402,32403,32404,32405,32406,32407,32408,32409,32410,32411,32412,32413,32414,32415,32416,32417,32418,32419,32420,32421,32422,32423,3

In [34]:
# To export directly in kaggle format
df_ = test.submission

#With BIO tags
"""label2id = {'O': 5, 'B-Dosage': 0, 'I-Dosage': 0, 'B-Drug': 1, 'I-Drug': 1, 
'B-Duration': 2, 'I-Duration': 2, 'B-Form': 3, 'I-Form': 3, 'B-Frequency': 4, 
'I-Frequency': 4, 'B-Route': 6, 'I-Route': 6, 'B-Treatment': 7, 'I-Treatment': 7}"""

label2id = {'O': 5, 'Dosage': 0, 'Drug': 1, 
'Duration': 2, 'Form': 3, 'Frequency': 4, 'Route': 6, 'Treatment': 7}


df_.label = df_.label.apply(lambda x: label2id[x])

df_1 = df_[['TokenId', 'label']]
df_1.columns = ['TokenId', 'Predicted']
df_1.set_index('TokenId').to_csv('scripts/final_prediction_v5.csv')
test.submission.to_csv('scripts/sylvain_aigris.csv')